In [1]:
import os


from coinapy.fetcher import get_exchange_symbols, get_ohlcv

True


/Users/doyeon/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


- Upbit Withdrawl Fee: https://upbit.com/service_center/guide 
    - http://upbit.com/service_center/notice?id=4446

In [2]:
from coinapy.util import find_concat_candidates, concat_csv

PERIODS = ["5SEC"]  # 5초, 1분 단위 데이터

exchanges = [  # https://coinmarketcap.com/rankings/exchanges/
    "BINANCE", "COINBASE", "OKEX", "CRYPTOCOM", "BITGET",
    "KRAKEN", "KUCOIN", "MEXC", "GATEIO", "BYBIT", "HTX", "GATEIO",
    "BITFINEX", "HUOBI", "BITMART", "LBANK"
]  # "BYBIT"

concat_candidate = list(filter(
    lambda x: x,
    (
        find_concat_candidates(x, doSort=False)
        for x in (
            f"{y}/{symbol}"
            for y in (
                f"data/{period}/{exc}"
                for period in PERIODS
                for exc in exchanges
                if os.path.exists(f"data/{period}/{exc}")
            )
            for symbol in os.listdir(y)
        )
        if os.path.exists(x)
    )
))

for x in concat_candidate:
    concat_csv(*x)

In [3]:
currency_order = ['USD', 'USDT', 'USDC', 'KRW', 'EUR', 'CNY', 'JPY']
# GBP


# target = "BTC"
# targets = [
#     "BTC", "ETH", "XRP", "SOL", "TRX", "ADA", "DOGE", "BNB",
#     "USDT", "USDC"
#     # https://coinmarketcap.com/
# ]
targets = [  # https://coinmarketcap.com/view/stablecoin/
    'DAI', 'USDE', "USDC", "USDT", "FDUSD", "USD0", "PYUSD", "FRAX",
    "TUSD", "USDY"

]

# targets = [
#     "LINK",
# ]

In [4]:
from coinapy.util import create_folders, kst_to_utc, look_symbols_in_exchange


print(kst_to_utc("2025-03-05T21:15:00.010000+09:00"))
# look_symbols_in_exchange("BINANCE")

2025-03-05T12:15:00.0100000Z


In [5]:
time_start = kst_to_utc("2024-12-01T22:00:00.010000+09:00")
time_end = kst_to_utc("2024-12-06T22:00:00.010000+09:00")
limit = 100000

In [6]:
from coinapy.util import aggregate_data


fnames = []

for target in targets:
    symbols = get_exchange_symbols(
        target, exchanges,
        types=["spot"],
        currency=currency_order,
    )

    if not symbols:
        print(target, ": No Symbols")
        continue

    create_folders(PERIODS, symbols)
    print(symbols)

    for exchange, symbol_id in symbols.items():
        print(f"📌 {exchange} {target} symbol_id: {symbol_id}")
        for s in (s for s in symbol_id if s):
            sym_id = "_".join(s.split("_")[1:])
            for period in PERIODS:
                filename = f"data/{period}/{exchange}/{sym_id}/{time_start}_{time_end}.csv"
                if os.path.exists(filename):
                    print(f"📂 {s} already exists")
                    fnames.append(filename)
                    continue

                data = get_ohlcv(s, period, time_start, time_end, limit=limit)
                if data.empty:
                    continue

                data.to_csv(filename, index=False)
                fnames.append(filename)

{'GATEIO': ['GATEIO_SPOT_DAI_USDT', 'GATEIO_SPOT_DAI_USD'], 'BITFINEX': ['BITFINEX_SPOT_DAI_USD'], 'KRAKEN': ['KRAKEN_SPOT_DAI_USDT', 'KRAKEN_SPOT_DAI_USD', 'KRAKEN_SPOT_DAI_EUR', 'KRAKEN_SPOT_DAI_USDT_57CC80'], 'OKEX': ['OKEX_SPOT_DAI_USDT', 'OKEX_SPOT_DAI_USDC', 'OKEX_SPOT_DAI_USD'], 'COINBASE': ['COINBASE_SPOT_DAI_USD', 'COINBASE_SPOT_DAI_USDC'], 'CRYPTOCOM': ['CRYPTOCOM_SPOT_DAI_USDT', 'CRYPTOCOM_SPOT_DAI_USD', 'CRYPTOCOM_SPOT_DAI_USDC'], 'BITMART': ['BITMART_SPOT_DAI_USDT', 'BITMART_SPOT_DAI_USDC'], 'BITGET': ['BITGET_SPOT_DAI_USDT', 'BITGET_SPOT_DAI_USDC'], 'MEXC': ['MEXC_SPOT_DAI_USDT'], 'BINANCE': ['BINANCE_SPOT_DAI_USDT'], 'HUOBI': ['HUOBI_SPOT_DAI_USDT']}
📌 GATEIO DAI symbol_id: ['GATEIO_SPOT_DAI_USDT', 'GATEIO_SPOT_DAI_USD']
📂 GATEIO_SPOT_DAI_USDT already exists
Empty Data
📌 BITFINEX DAI symbol_id: ['BITFINEX_SPOT_DAI_USD']
📂 BITFINEX_SPOT_DAI_USD already exists
📌 KRAKEN DAI symbol_id: ['KRAKEN_SPOT_DAI_USDT', 'KRAKEN_SPOT_DAI_USD', 'KRAKEN_SPOT_DAI_EUR', 'KRAKEN_SPOT_DAI_US

In [7]:
os.makedirs("output", exist_ok=True)
output_fname = "output/martial-law-5day-5SEC_stablecoin.csv"
df = aggregate_data(
    output_fname,
    PERIODS, exchanges,
    [
        f"{time_start}_{time_end}",
    ]
)

print(f"📊 {output_fname}")

📊 output/martial-law-5day-5SEC_stablecoin.csv


In [13]:
# aggregate_data(
#     PERIODS, exchanges, time_start + "_" + time_end
# )